In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import base64
import seaborn               as sns
import folium  #needed for interactive map
from folium.plugins import HeatMap
%matplotlib inline
sns.set()
from sklearn.linear_model import LinearRegression

In [2]:
#opening the data file (Food Deserts)

with open('Data/Cleaned Food Desert Data.csv') as f:
    df = pd.read_csv(f)
    
    
#opening second the data file (Health Outcomes)

with open('Data/Cleaned Health Outcomes Data.csv') as g:
    df2 = pd.read_csv(g)

In [3]:
#Renaming the census tract Identifier column in the health dataset in order to merge
df2 = df2.rename(columns={"tract_fips": "census_tract"})

In [4]:
#CREATING OUR COMBINED DATASET "cdf"
cdf = pd.merge(df, df2, on="census_tract")

In [5]:
#printing the head and shape of the combined dataset
print(cdf.head())
print(cdf.shape)

   census_tract    state   county  urban  population_2010  \
0    1001020100  Alabama  Autauga      1             1912   
1    1001020200  Alabama  Autauga      1             2170   
2    1001020300  Alabama  Autauga      1             3373   
3    1001020400  Alabama  Autauga      1             4386   
4    1001020500  Alabama  Autauga      1            10766   

   occupied_housing_units_2010  li_la_tracts_1_and_10_mile  \
0                          693                           0   
1                          743                           0   
2                         1256                           0   
3                         1722                           0   
4                         4082                           0   

   li_la_tracts_1/2_and_10_mile  li_la_tracts_1_and_20_mile  \
0                             0                           0   
1                             0                           0   
2                             0                           0   
3       

In [6]:
#Removing non-urban tracts: creating a series of the non-urban tracts
non_urban = cdf.index[cdf['urban'] == 0]

#making the series a list, dropping the list form 'cdf'
non_urban_list = non_urban.tolist()
cdf = cdf.drop(cdf.index[non_urban_list])

#printing the different groups in the "urban" column to confirm it's only urban tracts
cdf['urban'].groupby(cdf['urban']).value_counts()

urban  urban
1      1        53182
Name: urban, dtype: int64

**What data we're keeping:**
state_abbr = State abbreviation

county_fips = 5 digit zip code

total_population = 2010 census population count

census_tract = Census tract FIPS code

diabetes_crude_prev = Model-based estimate for crude prevalence of diagnosed diabetes among adults aged >=18 years, 2019

diabetes_crude95cI = Estimated confidence interval for crude prevalence of diagnosed diabetes among adults aged >=18 year

geolocation = Latitude, longitude of city centroid

occupied_housing_units_2010 = The count of occupied housing units in the tract as of 2010

_____

Multiple flag variables idenitfying tracts as low-income (li), low-access (la), or both.

Multiple variables for the counts and shares of low-income and low-access populations by distance (e.g. la_pop_1 is low access population count at 1 mile)

Multiple variables for the counts and shares of the low-access population by race by distance (e.g. la_black_1 and la_black_1_share are the population count and share of the low-access black population at 1 mile).


Multiple variables for the counts andd shares of the housing units without vehicles which would be considered low-access by distance (e.g. lahunv10 would be Housing units without vehicle count beyond 10 miles from supermarket.

Multiple measures for weatlh (e.g. Poverty rate, Median Family income)


___
**What data we're dropping:**
Multiple variables relating to measures beyonnd a 20 mile marker since these are used to track non-urban tracts which we have dropped to focus on urban tracts. 

Multiple variables for the counts and shares of the low-access kid and senior population by distance.

In [7]:
#checking shape of dataframe before dropping unnecessary variables
cdf.shape

# Removing the columns we no longer need
cdf = cdf.drop(columns=['li_la_tracts_1_and_20_mile',
                  'low_access_1_and_20_mile',
                  'low_access_tracts_20_mile',
                  'la_tracts_vehicle_20_mile',
                  'la_population_1_and_20',
                  'la_low_income_1_and_20',
                  'la_kids_1/2',
                  'la_kids_1/2_share',
                  'la_seniors_1/2',
                  'la_seniros_1/2_share',
                  'la_kids_10',
                  'la_kids_10_share',
                  'la_seniros_10',
                  'la_seniors_10_share',
                  'la_population_20',
                  'la_population_20_share',
                  'la_low_income_20',
                  'la_low_income_20_share',
                  'la_kids_20',
                  'la_kids_20_share',
                  'la_seniors_20',
                  ';a_seniros_20_share',
                  'la_white_20',
                  'la_white_20_share',
                  'la_black_20',
                  'la_black_20_share',
                  'la_asian_20',
                  'la_asian_20_share',
                  'la_nhopi_20',
                  'la_nhopi_share',
                  'la_aian_20',
                  'la_aian_20_share',
                  'la_multi_race_20',
                  'la_multi_race_20_share',
                  'la_hisp_20',
                  'la_hisp_20_share',
                  'la_hu_nv_20',
                  'la_hu_nv_20_share',
                  'la_snap_20',
                  'la_snap_20_share',])


In [8]:
#checking to make sure it worked
cdf.shape

(53182, 106)

In [9]:
 #EXPORTING THE CLEANED COMBINED DATASET FOR FRONT END PURPOSES
cdf.to_csv('Master Dataset.csv', index= False)